In [3]:
# importing required packages
                                                
# importing of required libaraies
import nltk
import re
import numpy as np
from nltk import sent_tokenize,word_tokenize
from nltk.tokenize  import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.probability import FreqDist
from string import punctuation
from nltk.corpus import stopwords
from nltk.text import Text
from string import punctuation
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [24]:
# importing threat actor,malware,tool data
threat_actors = pd.read_csv(r'data/threat_actors.csv')
threat_actors['label'] = 0 # labelling threat actor as 0
tools = pd.read_csv(r'data/tool_raw.csv')
malware = pd.read_csv(r'data/malware_raw.csv')[1:100]

In [20]:
# function to merging  the malware and tool data set
def table_info_extract(df,label):
    df.fillna('',inplace=True)
    text=[]
    name=[]
    col = ['Name','Associated Software','Description']
    initial = True
    for i in col:
        if i=='Name':
            name.append(df[i].str.lower())
        if initial:
            a= df[i].str.lower()
            initial = False
        else:
            a = a+' '+df[i].str.lower()
            
    text.append(a)
    result = pd.DataFrame({'Name':name[0],'Description':text[0],'label':label})
    return result

In [21]:
#creating overaall training set
df=threat_actors[['Name','Description','label']]
df = df.append(table_info_extract(malware,1))
df = df.append(table_info_extract(tools,2))

In [23]:
df.to_csv('training_data.csv',index=False)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df=10, norm='l1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.Description).toarray()
labels = df.label
features.shape

(253, 904)

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(df['Description'], df['label'], random_state = 123)
count_vect = CountVectorizer(min_df=10,stop_words='english')
tfidf_transformer = TfidfTransformer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [42]:
count_vect.vocabulary_

{'threat': 359,
 'group': 147,
 'suspected': 313,
 'associated': 21,
 'chinese': 49,
 'government': 146,
 'shares': 293,
 'groups': 148,
 'command': 59,
 'line': 181,
 'interface': 168,
 'connection': 71,
 'proxy': 260,
 'data': 84,
 'obfuscation': 223,
 'disabling': 98,
 'security': 286,
 'tools': 365,
 'exfiltration': 133,
 'control': 75,
 'channel': 46,
 'file': 139,
 'directory': 97,
 'discovery': 99,
 'modify': 207,
 'registry': 267,
 'multi': 210,
 'stage': 305,
 'channels': 47,
 'process': 256,
 'query': 263,
 'run': 274,
 'keys': 172,
 'startup': 308,
 'folder': 142,
 'shortcut': 296,
 'modification': 205,
 'standard': 307,
 'application': 19,
 'layer': 177,
 'protocol': 258,
 'information': 161,
 'encrypted': 119,
 'local': 189,
 'removable': 271,
 'media': 199,
 'staged': 306,
 'custom': 82,
 'fallback': 137,
 'cryptographic': 81,
 'non': 220,
 'active': 10,
 'activity': 11,
 'connections': 72,
 'networks': 218,
 'actors': 12,
 'spearphishing': 303,
 'enterprise': 120,
 't111

In [43]:
X_test_tfidf =count_vect.transform(X_test)
y_pred = clf.predict(X_test_tfidf)
accuracy_score(y_test, y_pred)

0.921875

In [46]:
X_train_tfidf

<189x394 sparse matrix of type '<class 'numpy.float64'>'
	with 9415 stored elements in Compressed Sparse Row format>

In [31]:
from sklearn.metrics import confusion_matrix

In [32]:
confusion_matrix(y_test, y_pred)

array([[22,  0,  0],
       [ 4, 21,  0],
       [ 9,  5,  3]], dtype=int64)

In [21]:
X_test=['''HTRAN is a tool that proxies connections through intermediate hops and aids users in disguising their true geographical location. It can be used by adversaries to hide their location when interacting with the victim networks.''']

In [22]:
X_test_tfidf =count_vect.transform(X_test)
y_pred = clf.predict(X_test_tfidf)

In [23]:
y_pred

array([2], dtype=int64)

In [ ]:
for i in df.Description[45]:
    print(i)

In [ ]:
y_pred

In [48]:
from numpy import count_nonzero

In [50]:
sparsity = 1.0 - ( count_nonzero(X_train_tfidf.toarray()) / float(X_train_tfidf.toarray().size) )
print(sparsity)

0.8735664598608761


In [ ]:
count_nonzero(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Description'], df['label'], random_state = 1)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from yellowbrick.classifier import ClassificationReport

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

nb.fit(X_train, y_train)


y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=df.label.unique()))

In [ ]:
df.label.unique()

In [ ]:
def text_preprocessing(df):
    
    for index,row in df.iterrows():
        
        text = row['Description']
        words = [w for sent in nltk.sent_tokenize(str(text)) for w in nltk.word_tokenize(sent)]    

In [ ]:
text = df['Description'][0]
words = [w for sent in nltk.sent_tokenize(str(text)) for w in nltk.word_tokenize(sent)]    